# Segunda analise dos dados dos ativos americanos

#### 1. Carregando os mesmos dados da primeira analise

In [117]:
# Carregando bibliotecas necessárias
import pandas as pd
import yfinance as yf
import plotly.express as px
from plotly import graph_objects as go
from plotly.graph_objs import Layout

In [118]:
# Carregando os dados
tickers = ['AAPL', 'AMZN', 'DIS']
start_date = "2021-09-19"
end_date = "2023-09-19"
yf = yf.download(tickers, start=start_date, end=end_date)
yf.head()

[*********************100%%**********************]  3 of 3 completed

Adj Close                               Close              \
                  AAPL        AMZN         DIS        AAPL        AMZN   
Date                                                                     
2021-09-20  141.305222  167.786499  178.610001  142.940002  167.786499   
2021-09-21  141.789612  167.181503  171.169998  143.429993  167.181503   
2021-09-22  144.181931  169.002502  173.649994  145.850006  169.002502   
2021-09-23  145.150711  170.800003  176.250000  146.830002  170.800003   
2021-09-24  145.239716  171.276001  176.000000  146.919998  171.276001   

                              High                                 Low  \
                   DIS        AAPL        AMZN         DIS        AAPL   
Date                                                                     
2021-09-20  178.610001  144.839996  170.949997  180.880005  141.270004   
2021-09-21  171.169998  144.600006  168.985001  182.229996  142.779999   
2021-09-22  173.649994  146.429993  169.449997  175.339996  143.699997   
2021-09-23  176.250000  147.080002  171.447998  177.190002  145.639999   
2021-09-24  176.000000  147.470001  171.462997  176.699997  145.559998   

                                          Open                          \
                  AMZN         DIS        AAPL        AMZN         DIS   
Date                                                                     
2021-09-20  165.250504  176.550003  143.800003  169.800003  179.740005   
2021-09-21  166.619507  169.029999  143.929993  168.750000  180.320007   
2021-09-22  167.052505  171.660004  144.449997  167.550003  172.940002   
2021-09-23  169.002502  174.369995  146.649994  169.002502  174.940002   
2021-09-24  169.669998  175.080002  145.660004  170.100494  175.300003   

               Volume                      
                 AAPL      AMZN       DIS  
Date                                       
2021-09-20  123478900  93382000   8056600  
2021-09-21   75834000  55618000  23216000  
2021-09-22   76404300  48228000  14214500  
2021-09-23   64838200  47588000   8052900  
2021-09-24   53477900  42324000   5712800

#### 2. Média móvel

1. Fazendo uma média móvel em 90 dias dos valores de fechamento ajustados dos ativos.
2. Utilizando a fução ```rolling()``` e calculando o desvio pardão desses ativos ao longo do tempo.
2. Montando um gráfico de linhas para as médias móveis dos ativos da base
4. Montando um gráfico de linhas para os desvios padrão em janela móvel
5. Comparando os dois gráficos feitos com o da primeira analise. Vendo se eles apontam conclusões semelhantes quanto aos períodos de maior volatilidade (variação) dos ativos.

## 1

In [119]:
# Calculando a media movel dos ativos ao longo de 90 dias
yf_retornos = yf['Adj Close'].pct_change()

yf_retornos_mm90 = yf_retornos.rolling(90).mean()
yf_retornos_mm90_vertical = yf_retornos_mm90.stack().reset_index()
yf_retornos_mm90_vertical.columns = ['data', 'ativo', 'valor']
yf_retornos_mm90_vertical.head()

,data,ativo,valor
0,2022-01-27,AAPL,0.001325
1,2022-01-27,AMZN,-0.001897
2,2022-01-27,DIS,-0.002919
3,2022-01-28,AAPL,0.002062
4,2022-01-28,AMZN,-0.001512


## 2

In [120]:
# Calculando o desvio padrão dos ativos ao longo do tempo com uma janela móvel de 90 dias
yf_desvio_padrao_mm90 = yf_retornos.rolling(90).std()
yf_desvio_padrao_mm90_vertical = yf_desvio_padrao_mm90.stack().reset_index()
yf_desvio_padrao_mm90_vertical.columns = ['data', 'ativo', 'desvio_padrao']


## 3

In [121]:
# Criando o gráfico de linha usando o Plotly para a media movel de 90 dias
fig = px.line(yf_retornos_mm90_vertical, x='data', y='valor', color='ativo',
              labels={'Date': 'Data', 'value': 'Valores'},
              title='Media movel de 90 dias',
              template='simple_white')

# Exibindo o gráfico
fig.show()

## 4

In [122]:
# Criando o gráfico de linha para o desvio padrão de 90 dias
fig = px.line(yf_desvio_padrao_mm90_vertical, x='data', y='desvio_padrao', color='ativo',
                            labels={'data': 'Data', 'desvio_padrao': 'Desvio Padrão'},
                            title='Desvio Padrão de 90 dias',
                            template='simple_white')

# Exibindo o gráfico de desvio padrão
fig.show()

## 5

Gráfico da primeira análise para comparação

In [123]:
# Calculando a amplitude de variação dentro do dia e adicionando uma coluna chamada "Amplitude" para cada ação
yf['AAPL_Amplitude'] = yf['High']['AAPL'] - yf['Low']['AAPL']
yf['AMZN_Amplitude'] = yf['High']['AMZN'] - yf['Low']['AMZN']
yf['DIS_Amplitude'] = yf['High']['DIS'] - yf['Low']['DIS']

# Criando um DataFrame separado com as colunas de amplitude para cada ativo
amplitude_yf = yf[['AAPL_Amplitude', 'AMZN_Amplitude', 'DIS_Amplitude']]

# Resetando o índice para usar a coluna 'Date' como x-axis no gráfico
amplitude_yf.reset_index(inplace=True)

# Criando o gráfico de linha usando o Plotly
fig = px.line(amplitude_yf, x='Date', y=['AAPL_Amplitude', 'AMZN_Amplitude', 'DIS_Amplitude'],
              labels={'Date': 'Data', 'value': 'Amplitude de Variação'},
              title='Variação da Amplitude de Variação ao Longo do Tempo',
              template='simple_white')

# Exibindo o gráfico
fig.show()

#### 3. Gráfico de *candlestick*
1. Selecionando o período dos últimos 60 dias (corridos) para um dos ativos.
2. Montando um gráfico de *candlestick* para esses dados junto com a media movel de 60 dias.

In [124]:
# Criando uma lista com os nomes dos indicadores que desejamos extrair
lista_indicadores = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

# Criando uma lista com o nome do ativo 'AMZN' para cada indicador na lista
ativo = ['AMZN'] * len(lista_indicadores)

# Mesclando a lista de indicadores com o nome do ativo correspondente
colunas = list(zip(lista_indicadores, ativo))

# Selecionando os dados do ativo 'AMZN' a partir de 19 de setembro de 2021 até o final do conjunto de dados
# Usando as colunas especificadas na lista 'colunas' como guia
amzn = yf.loc['06/22/2023':, colunas]

# Definindo os nomes das colunas do DataFrame 'amzn' com os nomes da lista de indicadores
amzn.columns = lista_indicadores


In [125]:

# Calculando a média móvel de 6 dias
amzn["MA6"] = amzn["Close"].rolling(6).mean()

# Criando o gráfico
graph = {
    'x': amzn.index,
    'open': amzn.Open,
    'close': amzn.Close,
    'high': amzn.High,
    'low': amzn.Low,
    'type': 'candlestick',
    'name': 'AMZN',
    'showlegend': True
}

ma6 = go.Scatter(
    x=amzn.index,
    y=amzn["MA6"],
    mode="lines",
    line=dict(color="blue", width=1),
)

layout = go.Layout(
    paper_bgcolor = 'rgba (0, 0,0,0)',
    plot_bgcolor = 'rgba (0,0,0, 0)'
)

grafico = go.Figure(
    data = [graph, ma6],
    layout_title = "Amazon",
    layout=layout
)
grafico.update_xaxes (showgrid=True, gridwidth=1, gridcolor='LightGrey')
grafico.update_yaxes (showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Exibindo o gráfico
grafico.show()
